In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_UCI as uci
import tasker_link_prediction as t_lp
from splitter import splitter
from models import Baseline_node2vec
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_UCI_baseline.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/UCI_N2V_baseline/',
 'classifier_hidden_size': 24,
 'ffn_fusion_size': 12,
 'ffn_hidden_size': 12,
 'gcn_fusion_size': 24,
 'spatial_hidden_size': 24,
 'spatial_input_dim': 64,
 'temporal_hidden_size': 24,
 'temporal_input_dim': 60,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.71,
 'dev_proportion': 0.1,
 'data_path': 'data/UCI/',
 'prep_data_path': 'prep_data/UCI_neg/'}

In [4]:
data = uci.UC_Irvine_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= Baseline_node2vec(config['spatial_input_dim'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])
print(trainer.patience)

Dataset splits sizes:  train 53 dev 9 test 17
10


In [5]:
trainer.train(config['num_epochs'])

Epoch 1, Step 1, Loss: 0.10255007445812225
Epoch 1, Step 2, Loss: 0.0997573658823967
Epoch 1, Step 3, Loss: 0.07204446196556091
Epoch 1, Step 4, Loss: 0.052311886101961136
Epoch 1, Step 5, Loss: 0.051578495651483536
Epoch 1, Step 6, Loss: 0.05000925064086914
Epoch 1, Step 7, Loss: 0.05103997141122818
Epoch 1, Step 8, Loss: 0.04373118653893471
Epoch 1, Step 9, Loss: 0.06669381260871887
Epoch 1, Step 10, Loss: 0.03523719683289528
Epoch 1, Step 11, Loss: 0.02712569385766983
Epoch 1, Step 12, Loss: 0.034706879407167435
Epoch 1, Step 13, Loss: 0.03359895199537277
Epoch 1, Step 14, Loss: 0.029371801763772964
Epoch 1, Step 15, Loss: 0.03791135922074318
Epoch 1, Step 16, Loss: 0.045834414660930634
Epoch 1, Step 17, Loss: 0.03293456882238388
Epoch 1, Step 18, Loss: 0.03444503992795944
Epoch 1, Step 19, Loss: 0.022374775260686874
Epoch 1, Step 20, Loss: 0.03182318061590195
Epoch 1, Step 21, Loss: 0.02940092794597149
Epoch 1, Step 22, Loss: 0.030493443831801414
Epoch 1, Step 23, Loss: 0.025781016

In [6]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.14783813215445232
Validation Metric MAPs: 0.2278318243948477
Test Metric MRRs: 0.16334140958806345
Test Metric MAPs: 0.11934162018561566


(0.16334140958806345, 0.11934162018561566)